## Constraint-based Platformer Generation

this notebook shows a step-by-step guide to do Generate a Platformer level based on previously made ones

In [2]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

level = cv2.imread('images/SuperMarioBros2(J)-World2-2.png', cv2.IMREAD_)


In [3]:
print(level.shape)

(240, 4128, 3)


In [4]:
height, width, _ = level.shape

M = height
N = 16


tiles = [level[x:x+M,y:y+N,:] for x in range(0,height,M) for y in range(0,width,N)]
print(len(tiles))

258


In [5]:
def pHash(cv_image):
        imgg = cv2.cvtColor(cv_image, cv2.COLOR_BGR2GRAY);
        h=cv2.img_hash.pHash(imgg) # 8-byte hash
        pH=int.from_bytes(h.tobytes(), byteorder='big', signed=False)
        return pH

hashed_tiles = [pHash(tile) for tile in tiles]

unique_hashes = list(dict.fromkeys(hashed_tiles))


hash_int_dict = dict([(unique_hashes[i], i) for i in range(len(unique_hashes))])
hash_tile_dict = dict(zip(hashed_tiles, tiles))
int_tile_dict = dict([(hash_int_dict[i], hash_tile_dict[i]) for i in unique_hashes])
len(unique_hashes)


129

In [6]:
N = 2

hash_to_int = [hash_int_dict.get(item,item)  for item in hashed_tiles]
gram = [(hash_to_int*2)[i: i + N] for i in range(len(hash_to_int))]
print(len(gram))

258


In [7]:
sequence = {}
for pair in gram:
    if pair[0] not in sequence:
        sequence[pair[0]] = [pair[1]]
    elif pair[1] not in sequence[pair[0]]:
        sequence[pair[0]].append(pair[1])


print(sequence)

{0: [1, 32, 0, 118], 1: [2], 2: [2, 3, 6, 8, 12, 14, 26, 36, 43, 16, 47, 64, 25, 79, 83, 86, 104, 105, 107, 113], 3: [4, 20, 101], 4: [5, 61, 67, 109], 5: [2, 92], 6: [7], 7: [2], 8: [9, 42], 9: [10], 10: [11], 11: [2], 12: [13], 13: [14, 2], 14: [15, 19], 15: [16], 16: [17, 45], 17: [18], 18: [2], 19: [2], 20: [21], 21: [22], 22: [23], 23: [24], 24: [25], 25: [25, 2, 74], 26: [27, 30, 48, 26, 66, 2, 90, 97, 99], 27: [28, 17], 28: [29, 54], 29: [26], 30: [31, 47, 26], 31: [0], 32: [33, 32, 100], 33: [34], 34: [35], 35: [3], 36: [37], 37: [38, 77], 38: [39], 39: [40], 40: [41], 41: [42], 42: [11, 42, 127], 43: [44], 44: [2], 45: [46], 46: [2], 47: [26, 2, 3], 48: [49], 49: [50], 50: [51], 51: [52], 52: [53], 53: [30], 54: [55], 55: [56], 56: [57], 57: [58], 58: [58, 59], 59: [60], 60: [47], 61: [62], 62: [63], 63: [13], 64: [65], 65: [42], 66: [26], 67: [68], 68: [69], 69: [70], 70: [71], 71: [72, 88], 72: [73], 73: [2], 74: [75], 75: [76], 76: [37], 77: [78, 103], 78: [0], 79: [80], 80

In [8]:
from numpy.random import randint, choice
def propagation(seq, starting_tile=None, size=10) -> list:
    level = []
    if starting_tile == None:
        starting_tile = randint(low=0, high=list(seq.keys())[-1])
    level.append(starting_tile)
    current_tile = starting_tile
    for i in range(size):
        next_tile = choice(seq[current_tile])
        level.append(next_tile)
        current_tile=next_tile
    return(level)
    

In [9]:
level_made = propagation(sequence, starting_tile=0, size=200)

level_to_tiles = [int_tile_dict[i] for i in level_made]

In [10]:
compiled_level = cv2.hconcat(level_to_tiles)

In [11]:
cv2.imshow("fixed", compiled_level)

cv2.waitKey(0)
cv2.destroyAllWindows()